In [1]:
#importa todas as bibliotecas usadas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets

In [4]:
#todas as turmas de 2013 até 2017
turma_periodo = ['dados/turmas-2013.1.csv','dados/turmas-2013.2.csv','dados/turmas-2014.1.csv','dados/turmas-2014.2.csv','dados/turmas-2015.1.csv',
                 'dados/turmas-2015.2.csv','dados/turmas-2016.1.csv', 'dados/turmas-2016.2.csv', 'dados/turmas-2017.1.csv', 'dados/turmas-2017.2.csv',]

In [6]:
#le todos os dados de todos os arquivos e salva em uma conjunto de datasets
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma','id_componente_curricular', 'ano', 'nivel_ensino', 'periodo','situacao_turma']) for periodo in turma_periodo]

In [7]:
#concatena os data sets
df_turmas = pd.concat(frames)
df_turmas.head()

,id_turma,id_componente_curricular,nivel_ensino,ano,periodo,situacao_turma
0,1196178,50280.0,LATO SENSU,2013,1.0,CONSOLIDADA
1,1196210,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
2,1196211,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
3,1196212,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
4,1196213,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA


In [8]:
# filtrar apenas disciplinas com código "CONSOLIDADA"
df_turmas_consolidadas = df_turmas[df_turmas.situacao_turma == 'CONSOLIDADA']
df_turmas_consolidadas.dropna(inplace=True)
df_turmas_consolidadas.head()

/home/cinthia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id_turma,id_componente_curricular,nivel_ensino,ano,periodo,situacao_turma
0,1196178,50280.0,LATO SENSU,2013,1.0,CONSOLIDADA
1,1196210,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
2,1196211,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
3,1196212,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA
4,1196213,52540.0,TÉCNICO,2013,1.0,CONSOLIDADA


In [9]:
#muda o nome da coluna do componente e muda o tipo para int64 - para fazer o merge com as disciplinas
df_turmas_consolidadas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
df_turmas_consolidadas.id_componente = df_turmas_consolidadas.id_componente.astype(np.int64)

/home/cinthia/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/home/cinthia/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
#armazena os dados das disciplinas
disciplinas_url = 'dados/componentes-curriculares-presenciais.csv'
df_disciplinas = pd.read_csv(disciplinas_url, sep=';',usecols=["id_componente","codigo","nome"])
df_disciplinas.dropna(inplace=True)
df_disciplinas.head()

,id_componente,codigo,nome
0,18643,MUT508,COMPOSIÇÃO EM MÚSICA POPULAR III
1,18644,MUT516,CO-REPETIÇÃO
2,18645,MUT520,CONTRABAIXO ACÚSTICO I
3,18646,MUT165,PIANO COMPLEMENTAR I
4,18647,MUT167,PIANO COMPLEMENTAR III


In [11]:
#filtra apenas as disciplinas do imd
disciplinas_imd = df_disciplinas[df_disciplinas.codigo.str.contains("IMD")]
disciplinas_imd.head()

,id_componente,codigo,nome
20970,51954,IMD0001,INOVAÇÃO TECNOLÓGICA
21229,52247,IMD0002,INOVAÇÃO TECNOLÓGICA
21230,52248,IMD0003,PROCESSO E QUALIDADE DE SOFTWARE
21231,52249,IMD0004,GERÊNCIA DE PROJETOS
21371,52390,IMD0005,INOVAÇÃO TECNOLÓGICA


In [14]:
#merge entre turmas e disciplinas
colunas = ["id_turma","id_componente","codigo","nome","nivel_ensino","ano","periodo","situacao_turma"]
turmas_disciplinas = pd.merge(df_turmas_consolidadas, disciplinas_imd, on='id_componente')[colunas]
turmas_disciplinas = turmas_disciplinas[~turmas_disciplinas.id_turma.duplicated()]
turmas_disciplinas.head()

,id_turma,id_componente,codigo,nome,nivel_ensino,ano,periodo,situacao_turma
0,1196210,52540,IMD0901,INTRODUÇÃO ÀS TECNOLOGIAS DA INFORMAÇÃO,TÉCNICO,2013,1.0,CONSOLIDADA
1,1196211,52540,IMD0901,INTRODUÇÃO ÀS TECNOLOGIAS DA INFORMAÇÃO,TÉCNICO,2013,1.0,CONSOLIDADA
2,1196212,52540,IMD0901,INTRODUÇÃO ÀS TECNOLOGIAS DA INFORMAÇÃO,TÉCNICO,2013,1.0,CONSOLIDADA
3,1196213,52540,IMD0901,INTRODUÇÃO ÀS TECNOLOGIAS DA INFORMAÇÃO,TÉCNICO,2013,1.0,CONSOLIDADA
4,1196214,52540,IMD0901,INTRODUÇÃO ÀS TECNOLOGIAS DA INFORMAÇÃO,TÉCNICO,2013,1.0,CONSOLIDADA


In [15]:
# armazena as avaliações dos professores da ufrn
avaliacao = pd.read_csv("dados/avaliacaodocencia.csv", sep=';')
avaliacao.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
0,15370,FERNANDO MOREIRA DA SILVA,1209923,2013,2,14,9.17,1.34,8.13,2.70,7.61,2.53
1,15370,FERNANDO MOREIRA DA SILVA,1222008,2014,1,15,9.44,0.92,7.17,3.08,8.71,1.72
2,15370,FERNANDO MOREIRA DA SILVA,1222218,2014,1,38,9.19,1.67,6.77,3.29,8.12,2.53
3,15370,FERNANDO MOREIRA DA SILVA,1229520,2014,2,13,9.06,1.16,7.68,2.51,7.62,2.77
4,15370,FERNANDO MOREIRA DA SILVA,1229545,2014,2,24,9.27,1.28,8.45,1.72,8.48,1.86


In [34]:
colunas = ["id_turma","id_componente","codigo","nome","nivel_ensino","nome_docente","postura_profissional_media","postura_profissional_DP","atuacao_profissional_media","atuacao_profissional_DP","autoavaliacao_aluno_media","autoavaliacao_aluno_DP"]
av_imd = pd.merge(turmas_disciplinas, avaliacao, on='id_turma')[colunas]
lista_prof = av_imd.nome_docente.unique()
lista_prof.sort()
av_imd.head()

,id_turma,id_componente,codigo,nome,nivel_ensino,nome_docente,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
0,1210347,52656,IMD0014,INTRODUÇÃO AO CÁLCULO,GRADUAÇÃO,SAMYR SILVA BEZERRA JACOME,9.19,2.02,8.51,2.53,7.48,2.63
1,1211547,52656,IMD0014,INTRODUÇÃO AO CÁLCULO,GRADUAÇÃO,SAMYR SILVA BEZERRA JACOME,9.72,0.66,8.82,2.09,7.96,2.61
2,1210362,52651,IMD0012.0,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,GRADUAÇÃO,ADJA FERREIRA DE ANDRADE,9.08,1.44,7.35,3.02,8.73,2.19
3,1211551,52651,IMD0012.0,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,GRADUAÇÃO,ADJA FERREIRA DE ANDRADE,9.63,0.87,8.70,2.18,8.07,2.72
4,1217933,52651,IMD0012.0,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,GRADUAÇÃO,ANDRE MAURICIO CUNHA CAMPOS,9.69,0.60,8.92,1.37,8.61,1.81


In [40]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = av_imd.loc[av_imd['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof,);

interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDERSON PAIVA CRUZ', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE LUIZ DE SOUZA BRITO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JORGE DE MACEDO', 'BRUNO SANTANA DA SILVA', 'CARLA DA COSTA FERNANDES CURVELO', 'CARLOS AUGUSTO PROLO', 'CARLOS EDUARDO DA SILVA', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANIEL SABINO AMORIM DE ARAUJO', 'DANIEL TEIXEIRA DOS SANTOS', 'DANILO CURVELO DE SOUZA', 'DAVID BORIS PAUL DEHARBE', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'ELIZABETH FERREIRA GOUVEA GOLDBARG', 'ELLIS MARIA SOUZA NORO', 'EMERSON MOURA DE ALENCAR', 'ERNANO ARRAIS JUNIOR', 'EUGENIO PACCELLI AGUIAR FREIRE', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'FERNANDA PALHANO XAVIER DE FONTES', 'FRANCISCO GEOCI DA SILVA', 'FREDERICO ARAUJO DA SILVA LOPES', 'GILIARD FAUSTINO DA SILVA', 'GUSTAVO GIRAO BARRETO DA SILVA', 'HADLEY MAGNO DA COSTA SIQUEIRA', 'HANDERSON BEZERRA MEDEIROS', 'HANIEL MOREIRA BARBOSA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO BATISTA DA COSTA JUNIOR', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JOSUÉ VITOR DE MEDEIROS JÚNIOR', 'JULIANA DE MELO PEREIRA', 'JULIO CESAR PAULINO DE MELO', 'KAYO GONCALVES E SILVA', 'KLEBER TAVARES FERNANDES', 'LEONARDO CESAR TEONACIO BEZERRA', 'LILIANE RIBEIRO DA SILVA', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'LUCIANA VIEIRA ANDRADE', 'MANOEL VERAS DE SOUSA NETO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MARCIO EDUARDO KREUTZ', 'MARJORY CRISTIANY DA COSTA ABREU', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'PATRICK CESAR ALVES TERREMATTE', 'PAULO ROBERTO BELTRAO MAIA', 'PRISCILLA TELES DE OLIVEIRA', 'RAFAEL BESERRA GOMES', 'RAFAEL VIDAL AROCA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'REGIVAN HUGO NUNES SANTIAGO', 'RENATO QUARENTEI GARDIMAN', 'RODRIGO TEIXEIRA RAMOS', 'SAMYR SILVA BEZERRA JACOME', 'SEBASTIAN YURI CAVALCANTI CATUNDA', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'TAMYRIS REZENDE FERREIRA', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VANESSA DANIELLE SANTOS FERREIRA', 'WAGNER GUEDES KERLLER', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))

In [36]:
% matplotlib inline
from ipywidgets import interact,interactive
import matplotlib.pyplot as plt
import numpy as np

def evolucaoGrafico(x):
    dados_prof = avaliacao.loc[avaliacao['nome_docente'] == x]
    g = sns.kdeplot(dados_prof["postura_profissional_media"], dados_prof["ano"])
    sns.despine(left=True, bottom=True)
    plt.show()
  
    
interactive_plot = interact(evolucaoGrafico, x=lista_prof,)

interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDERSON PAIVA CRUZ', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE LUIZ DE SOUZA BRITO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JORGE DE MACEDO', 'BRUNO SANTANA DA SILVA', 'CARLA DA COSTA FERNANDES CURVELO', 'CARLOS AUGUSTO PROLO', 'CARLOS EDUARDO DA SILVA', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANIEL SABINO AMORIM DE ARAUJO', 'DANIEL TEIXEIRA DOS SANTOS', 'DANILO CURVELO DE SOUZA', 'DAVID BORIS PAUL DEHARBE', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'ELIZABETH FERREIRA GOUVEA GOLDBARG', 'ELLIS MARIA SOUZA NORO', 'EMERSON MOURA DE ALENCAR', 'ERNANO ARRAIS JUNIOR', 'EUGENIO PACCELLI AGUIAR FREIRE', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'FERNANDA PALHANO XAVIER DE FONTES', 'FRANCISCO GEOCI DA SILVA', 'FREDERICO ARAUJO DA SILVA LOPES', 'GILIARD FAUSTINO DA SILVA', 'GUSTAVO GIRAO BARRETO DA SILVA', 'HADLEY MAGNO DA COSTA SIQUEIRA', 'HANDERSON BEZERRA MEDEIROS', 'HANIEL MOREIRA BARBOSA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO BATISTA DA COSTA JUNIOR', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JOSUÉ VITOR DE MEDEIROS JÚNIOR', 'JULIANA DE MELO PEREIRA', 'JULIO CESAR PAULINO DE MELO', 'KAYO GONCALVES E SILVA', 'KLEBER TAVARES FERNANDES', 'LEONARDO CESAR TEONACIO BEZERRA', 'LILIANE RIBEIRO DA SILVA', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'LUCIANA VIEIRA ANDRADE', 'MANOEL VERAS DE SOUSA NETO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MARCIO EDUARDO KREUTZ', 'MARJORY CRISTIANY DA COSTA ABREU', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'PATRICK CESAR ALVES TERREMATTE', 'PAULO ROBERTO BELTRAO MAIA', 'PRISCILLA TELES DE OLIVEIRA', 'RAFAEL BESERRA GOMES', 'RAFAEL VIDAL AROCA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'REGIVAN HUGO NUNES SANTIAGO', 'RENATO QUARENTEI GARDIMAN', 'RODRIGO TEIXEIRA RAMOS', 'SAMYR SILVA BEZERRA JACOME', 'SEBASTIAN YURI CAVALCANTI CATUNDA', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'TAMYRIS REZENDE FERREIRA', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VANESSA DANIELLE SANTOS FERREIRA', 'WAGNER GUEDES KERLLER', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))